# Setup

In [1]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

In [52]:
# set up assistant

from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# Open the file and read the contents
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# print(prompt)

my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
)

In [40]:


# create a thread
my_thread = client.beta.threads.create()

import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown (with no delimiters) and with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


# Initial File Upload and Prompt

In [5]:
# get the job description from GitHub
import time


file_path = 'temp_files/jobdescription.txt'
# infra engineer example
#!wget -O file_path https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# medical assistant example
!wget -O 'temp_files/jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

# infra engineer example
# !wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# medical assistant example
!wget -O 'temp_files/companyvalues.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
# time.sleep(5)


# Upload the file
file_company_values = client.files.create(
  file=open("temp_files/companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("temp_files/jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-20 13:44:28--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483 (2.4K) [text/plain]
Saving to: ‘temp_files/jobdescription.txt’

temp_files/jobdescr 100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-20 13:44:28 (6.63 MB/s) - ‘temp_files/jobdescription.txt’ saved [2483/2483]

--2023-12-20 13:44:28--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

# Priming the Pump

In [ ]:
# Refresh the context to ensure it has read the files
primer_values = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the company values file, using the company name.
            """, [file_job_description.id, file_company_values.id])
print(primer_values)

In [54]:
# Refresh the context to ensure it has read the files
primer_job = create_run_and_display_response(client,my_thread,my_assistant,"""
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id, file_company_values.id])
print(primer_job)


Values of Stanford Health Care - University Healthcare Alliance:

Innovation: Embracing innovation, continuous improvement, and revolutionizing care quality to shape the future of medicine.

Connection: Building trust, community, belonging, and shared purpose by caring for each other.

Teamwork: Restructuring collaboration across different roles and departments for greater impact and fulfillment.

Mentorship: Nurturing professional growth to ensure fulfilling and impactful careers.

Participation: Sharing diverse opinions and contributions to enhance decision-making and stakeholder engagement.

Action: Merging different perspectives and expertise to serve the common good.

---

Medical Assistant II

---

## Technical Competencies

1. Clinical Procedures
   - Performing clinical tasks and procedures with precision.
     - Venipuncture.
     - Electrocardiograms (ECGs).
     - Injections.
     - Vitals signs monitoring.

2. Computer Proficiency
   - Utilizing computer systems to enhance 

# Create Introduction

In [7]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"""
            Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.
            """)
print(introduction)

# Interview Playbook for Medical Assistant II

## Introduction

This Interview Playbook is designed for the hiring process of the Medical Assistant II position at Stanford Health Care - University Healthcare Alliance. The role is critical to ensure high-quality patient care and the efficient operation of healthcare services. Through this guide, the interview aims to objectively evaluate candidates on key technical and behavioral competencies that align with the company's values of Innovation, Connection, Teamwork, Mentorship, Participation, and Action. The ideal candidate will demonstrate proficiency in clinical procedures and healthcare software, effective patient communication, multitasking abilities, familiarity with emergency protocols, and embody an innovative mindset, cultural competence, team collaboration, a commitment to mentorship, and participatory decision-making.

## Interview Structure

The structure of the interview for the Medical Assistant II position is designed to pr

# Create Evaluation Criteria

In [8]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The evaluation of the candidate for the Medical Assistant II position will be based on the following competencies, which are crucial for success in the role at Stanford Health Care - University Healthcare Alliance. Interviewers are to use the scorecard provided below to rate the candidate on each competency and record relevant comments.

| Competency                        | Assessment (1-5) | Comments                      |
|-----------------------------------|------------------|-------------------------------|
| Clinical Procedures               |                  |                               |
| Healthcare Software Proficiency   |                  |                               |
| Patient Communication             |                  |                               |
| Multitasking Ability              |                  |                               |
| Emergency Protocols               |                  |                               |
| Innovative 

# Create Technical Assessment

In [9]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Evaluating a candidate's technical capabilities is critical to ensure they possess the necessary skills and knowledge for the Medical Assistant II position. The assessment focuses on their ability to perform clinical tasks, use healthcare software, communicate with patients, manage multiple responsibilities, and respond to emergency situations, in alignment with the expectations at Stanford Health Care - University Healthcare Alliance.

### Technical Competence 1: Clinical Procedures

_Candidates must demonstrate proficiency in various clinical tasks as expected in advanced medical assistance roles. Mastery of these procedures is essential for the delivery of high-quality patient care and safety._

#### Lead Question

"Can you describe an instance where you improved a clinical procedure in your last role, and how did it impact the quality of patient care provided?"

#### Probing Questions

- "How do you stay current with new clinical procedures relevant to your

In [55]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

In [11]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Patient Communication

_Patient communication is paramount for a Medical Assistant II. Candidates should exhibit strong abilities to engage with patients effectively, providing clear instructions and empathetic support while also ensuring proper documentation._

#### Lead Question

"Tell me about a time when you had to explain a complex medical procedure to a patient. How did you ensure they fully understood and felt comfortable with the information provided?"

#### Probing Questions

- "How would you handle a situation where a patient is upset or dissatisfied with their care?"
- "What methods do you employ when communicating sensitive information to patients?"
- "Could you describe how you document patient interactions and ensure all necessary details are captured?"
- "Give an example of how you've adapted your communication style to meet the needs of a diverse patient population."

#### Scenario

Imagine a scenario where a patient is anxious about an upcom

In [12]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Multitasking Ability

_The ability to multitask effectively is essential for a Medical Assistant II, requiring organized and efficient management of multiple responsibilities in a dynamic healthcare environment._

#### Lead Question

"Describe a day at your previous job where you had to juggle several tasks at once. How did you prioritize and ensure nothing was overlooked?"

#### Probing Questions

- "What tools or systems do you use to manage your daily tasks and ensure you meet all your responsibilities?"
- "How do you handle interruptions to your workflow, such as an unexpected patient issue or urgent request from a physician?"
- "Can you share an example of when you successfully completed a project or task under tight deadlines?"
- "What strategies do you employ to remain calm and focused when faced with multiple demands on your time and attention?"

#### Scenario

You have been tasked with preparing three patients for different appointments, processing 

In [13]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Emergency Protocols

_Candidates must be well-versed in emergency protocols, including Basic Life Support (BLS), to react effectively to clinical emergencies and ensure patient safety._

#### Lead Question

"Walk me through the steps you would take if you were the first to respond to a patient experiencing a cardiac emergency in the clinic."

#### Probing Questions

- "How often do you participate in emergency response training or drills, and what have you learned from them?"
- "Describe how you maintain composure and clarity of thought during an emergency."
- "Share an experience where your quick thinking and actions in an emergency made a difference in the outcome."
- "What is your process for documenting and debriefing after an emergency incident has occurred?"

#### Scenario

A patient in the waiting room suddenly becomes unconscious and unresponsive. Explain your immediate actions step-by-step, considering the protocols you must follow and how you would

In [14]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [15]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

The Behavioral Assessment evaluates how the candidate's personal attributes align with the company's core values such as Innovation, Connection, Teamwork, Mentorship, and Participation. These qualities ensure the candidate not only fits Stanford Health Care - University Healthcare Alliance's culture but also contributes positively to the work environment.

### Behavioral Competence 1: Innovative Mindset

_This competency assesses the candidate's openness to change and their proactive approach to seeking improvements in their scope of work, in alignment with Stanford Health Care - University Healthcare Alliance's value of Innovation._

#### Lead Question

"Can you give me an example of a time when you thought outside the box to solve a problem in your workplace?"

#### Probing Questions

- "What is your process for staying informed about the latest developments in patient care, and how do you apply this knowledge?"
- "How do you approach the introduction of new

In [16]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Cultural Competence

_Having a deep understanding and appreciation for various cultural perspectives is essential in healthcare. This competency ensures that the candidate can connect and effectively communicate with a diverse patient population._

#### Lead Question

"Tell me about a time when understanding a patient's cultural background helped you provide better care."

#### Probing Questions

- "How do you ensure you are sensitive to cultural differences in patient interactions?"
- "Describe a situation where you had to adjust your communication style to address cultural differences."
- "What strategies do you use to learn about and keep updated on the multicultural aspects of patient care?"
- "Can you share an example of how you collaborated with colleagues to address a cultural issue affecting patient care?"

#### Scenario

Imagine you're assisting a patient who has limited English proficiency, with cultural beliefs that differ significantly from conv

In [17]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Team Collaboration

_Teamwork is vital in a healthcare setting. This competency explores how well the candidate can collaborate with other healthcare professionals to achieve shared goals, reflecting Stanford Health Care - University Healthcare Alliance's value of Teamwork._

#### Lead Question

"Describe a time when working as part of a team allowed you to achieve better outcomes than if you had worked alone."

#### Probing Questions

- "How do you contribute to a positive team dynamic?"
- "What role do you typically play in a team setting, and can you give an example of how this has been effective?"
- "Discuss a situation where there was a disagreement within the team. How did you handle it?"
- "Can you describe an instance where you went above and beyond to support a team member?"

#### Scenario

You are part of a team tasked with implementing a new patient care initiative. The team has differing opinions on the approach to take. Explain how you would fa

In [18]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Mentorship and Growth

_This competency is about the willingness to engage in continuous learning, both as a mentor and a mentee, promoting professional development within the healthcare environment._

#### Lead Question

"Can you share an experience where you either mentored someone or were mentored by someone else, and the impact it had on your professional journey?"

#### Probing Questions

- "How do you approach opportunities for teaching and sharing knowledge with your colleagues?"
- "What strategies do you employ to continue your professional growth in the field of healthcare?"
- "As a mentor, how do you assess the needs of your mentee, and what has been your most successful outcome?"
- "Describe a time when receiving mentorship allowed you to enhance your skills or improve your performance."

#### Scenario

Consider you have been paired with a new Medical Assistant who is transitioning from another field of medicine into your specialty. Outline how y

In [19]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Decision-Making Participation

_Candidates should demonstrate an ability to effectively contribute to collective decision-making processes, valuing diverse perspectives and collaboration._

#### Lead Question

"Tell us about a time when you played a substantial role in making a decision within your team or organization. How did you ensure all voices were heard?"

#### Probing Questions

- "How do you balance between making swift decisions and ensuring all stakeholder opinions are considered?"
- "Can you describe an instance when you had to advocate for a less popular decision that you believed was right?"
- "What methods do you use to gather input from various team members, particularly those who might be less outspoken?"
- "Discuss a situation where you facilitated a team decision that led to improved patient care or service delivery."

#### Scenario

Imagine being part of a team meeting to decide on the best course of action for implementing a new clinica

In [20]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [21]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Medical Assistant II candidate at Stanford Health Care - University Healthcare Alliance is designed to assess the candidate's hands-on skills and proficiency in a real-world healthcare setting.

### Assignment Description:

The candidate will be presented with a simulated patient scenario that will require them to demonstrate a range of clinical procedures, use of the EMR system, patient communication, and multitasking under time constraints. The candidate will need to showcase their ability to prioritize tasks, respond to patient needs, navigate healthcare software, and adhere to emergency protocols effectively.

### Objectives:

1. Evaluate the candidate’s practical skills in performing essential medical assistant duties.
2. Test the candidate’s ability to use healthcare software in patient care context.
3. Observe the candidate’s communication with the simulated patient and colleagues.
4. Assess the candidate’s capacity to pr

# Create Team Interaction

In [22]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

The team fit section of the interview process is designed to gauge how well the Medical Assistant II candidate will integrate with the existing team at Stanford Health Care - University Healthcare Alliance. It provides insight into the candidate's ability to collaborate, communicate, and contribute to a team-oriented environment.

### Rationale:

Team interaction is key to the collaborative culture at Stanford Health Care - University Healthcare Alliance. It allows for the assessment of the candidate's interpersonal skills, their ability to work in a team setting, and how they handle real-time collaboration and decision-making.

### Team Interaction Activities:

- Group discussion on a relevant healthcare topic or common challenges faced in the clinic to see the candidate's input and engagement level.
- Role-playing a team meeting scenario where a new policy or procedure is being introduced, allowing observation of the candidate’s approach to the group dynamics.
- A brief 

# Create Candidate Q&A

In [23]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

The Candidate Q&A section provides an opportunity for the Medical Assistant II candidate to ask questions and gain a deeper understanding of Stanford Health Care - University Healthcare Alliance and the specific role for which they are interviewing. These questions reflect common inquiries candidates may have and offer comprehensive answers to facilitate an informative dialogue.

1. **Question**: Can you describe the typical day-to-day responsibilities for this Medical Assistant II role?
   
   **Answer**: As a Medical Assistant II, your day-to-day tasks will include performing clinical procedures, managing patient flow, documenting medical histories and vital signs in our EMR system, and providing patient education. You will need to multitask effectively, handle scheduling, and respond to emergency situations when they arise.

2. **Question**: How does Stanford Health Care - University Healthcare Alliance ensure continuing education and professional growth for its st

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [24]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('temp_files/playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [44]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [45]:
styles= """

   body {
        font-family: Source Sans Pro;
        font-size: 11pt;
        font-weight: 200;
        font-stretch: 75% 125%;
        color:#0D0D0D;
    }
    @page {
        margin: 0.65in;
        size: Letter;
        @bottom-right {
          content: counter(page);
          font-size: 11pt;
          color: #73706C;
        }
    }
    @page :first {
        @bottom-right {
          content: none;
        }
    }


    h1, h2, h3, h4 {
        font-family: Libre Baskerville;
        margin-top: 1em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 24pt;
        margin-top: 3em;
        page-break-after: avoid;
    }
    .intro h1{
        font-size: 50pt;
        margin-top: 3em;
        page-break-after: always;
    }
    @media print {
      /* Define styles for the cover page */
      .title {
        position: fixed;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: #F2F2F2; /* Change as needed */
        color: #0D0D0D; /* Change as needed */
        text-align: center;
        padding: 50px;
      }
    }
    .publication-date {
      position: absolute;
      bottom: 10px; /* adjust as needed */
      width: 100%;
      text-align: left;
      font-size: 11pt; /* adjust as needed */
      color: #73706C; /* adjust as needed */
    }


    h2 {
        font-size: 18pt;
        page-break-after: avoid;
    }
    h3 {
        font-size: 14pt;
        page-break-after: avoid;
    }
    h4 {
        font-size: 12pt;
        page-break-after: avoid;
    }
    p {
        margin-bottom: 1em;
    }


    h1, h2, h3, h4 {
        page-break-after: avoid;
    }

    p, table, figure {
        page-break-inside: avoid;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin-bottom: 1em;
        page-break-before: avoid;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
        font-size: 10pt;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    tr:nth-child(even) {{background-color: #f9f9f9;}}

    ul, ol {
        margin-bottom: 1em;
        margin-left: 0;
        padding-left:1.5em;
    }
    li {
        margin-bottom: 0.5em;
        margin-left: 0em;
    }

    .main {
      page-break-before: always;
    }

    .dotted-line {
        border-bottom: 1px dotted black;
        width: 100%;
        margin-top: 20px; /* This ensures spacing between each line */
        margin-bottom: 20px; /* Adjust if needed */
    }

    @media print {
        .dotted-line {
            margin-top: 20px; /* Adjust as needed for printing */
            margin-bottom: 20px; /* Adjust as needed for printing */
        }
    }

    .rating-container {
        display: flex;
        align-items: flex-end;
        margin-top: 20px;
        margin-bottom: 20px;
    }

    .rating-container label {
        margin-right: 10px;
        font-size: 10pt;
        font-family: Source Sans Pro;
    }

    .short-dotted-line {
        border-bottom: 1px dotted black;
        width: 150px; /* Adjust the width as needed */
        margin-left: 5px;
    }}

    @media print {
        .short-dotted-line {
            width: 150px; /* Adjust for printing if needed */
        }
    }
"""

## Create some useful HTML snippets

In [46]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [47]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class='intro'>
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a>.</div>
    {html_introduction}
    </section>
    <section class='main'>
    {html_job_description}
    </section>
    <section class='main'>
    {html_eval_criteria}
    </section>
    <section class='main'>
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('temp_files/playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [48]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment (1-5),Comments
Clinical Procedures,,
Healthcare Software Proficiency,,
Patient Communication,,
Multitasking Ability,,
Emergency Protocols,,
Innovative Mindset,,
Cultural Competence,,
Team Collaboration,,
Mentorship and Growth,,
Decision-Making Participation,,


# Create the Documents

Create the Word document

In [49]:
# Write HTML string to a file
with open('temp_files/playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s 'temp_files/playbook.html' -o temp_files/playbook.docx

# from google.colab import files
# files.download('playbook.docx')

Create the PDF

In [50]:
#!python3 --version
#!pango-view --version

#!brew install gobject-introspection
#!brew install gtk+3
#!brew install glade
#!brew install weasyprint

#!python3 -m venv venv
#!source venv/bin/activate
#%pip install cffi weasyprint
#!weasyprint --info

In [51]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('temp_files/playbook.html').write_pdf('temp_files/playbook.pdf')

# from google.colab import files
# files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')
